# __Tweepy: Busca de tweets com Twitter API V2__

Para a API V2 do Twitter temos dois _end points_ de busca de tweets:

    1. Busca recente, onde podemos coletar tweets dentro de uma janela de 7 dias;
    2. Busca completa, onde podemos coletar tweets desde de março de 2006

In [2]:
import tweepy, os, requests
import pandas as pd
import tweepy as tw
from datetime import datetime, timezone

In [4]:
keys = {
        'consumer_key': os.environ.get('API_KEY'),
        'consumer_secret': os.environ.get('API_SECRET'),
        'access_token_key': os.environ.get('ACCESS_TOKEN'),
        'access_token_secret': os.environ.get('ACCESS_TOKEN_SECRET'),
        'bearer_token': os.environ.get('BEARER_TOKEN')
    }

Para a API V2 não é necessário criar um autenticador, basta criar uma instância de _Client_ do Tweepy e colocar as seguintes informaçães para realizar a autenticação.

In [5]:
client = tw.Client(
    bearer_token=keys['bearer_token'],
    #keys['consumer_key'],
    #keys['consumer_secret'],
    #keys['access_token_key'],
    #keys['access_token_secret'],
    wait_on_rate_limit=True,
    #return_type=dict
)

## Busca Recente

In [6]:
tweets_recent = client.search_recent_tweets(
    query='python lang:pt', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency) 
)

In [8]:
#tweets = [tweet for tweet in tweets_recent['data']]
tweets = [tweet for tweet in tweets_recent.data]
len(tweets)

49

In [11]:
tweets[0]

<Tweet id=1567301404192120839 text='O meu professor segue firme tentando obrigar a turma a aprender Fortran, e disse que vai zerar se alguém entregar em Python'>

Mas só recupera esses dados do Tweet????

Para adicionar mais campos é necessário especificar na busca.

In [46]:
tweets_recent = client.search_recent_tweets(
    query='python lang:pt', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency)
    expansions=['author_id', 'geo.place_id', 'entities.mentions.username', 'referenced_tweets.id', 'in_reply_to_user_id', 'referenced_tweets.id.author_id'],
    place_fields=['full_name', 'country'],
    tweet_fields=['id', 'text', 'author_id', 'conversation_id', 'created_at', 'entities', 'in_reply_to_user_id', 'lang',
                   'public_metrics', 'referenced_tweets'])

O método _Cliente_ retorna um objeto _Response_, que é uma tupla de coleções, contendo _data_, _includes_, _errors_, e _meta_.

In [47]:
type(tweets_recent)

tweepy.client.Response

In [51]:
# acessar tweets
tweets_recent.data[:5] # também pode ser acessado dessa forma: tweets_recent[0]

[<Tweet id=1567301404192120839 text='O meu professor segue firme tentando obrigar a turma a aprender Fortran, e disse que vai zerar se alguém entregar em Python'>,
 <Tweet id=1567300131690610688 text='acho q vou virar essa noite aprendendo python 👌'>,
 <Tweet id=1567299170553274370 text='@bruxinhadoicmc e o papo de fazer em python?'>,
 <Tweet id=1567298678565511169 text='@x_heitor_x16 ano passado a gente tava estudando programação no técnico (a gente teve introdução pra python) \n\ndaí esse ano tá mais focado em desenvolvimento web msm'>,
 <Tweet id=1567296555933106176 text='@PaoCelestial_ct @HadokenEx pior que, em tão pouco tempo de aula, eu saberia fazer isso\né só saber o que cada coisa faz, o foda é que eu não sei que linguagem é essa, muito provavelmente é python'>]

In [55]:
# acessar dados do includes
tweets_recent.includes.keys() # um objeto que contém informações sobre os usuários e tweets

dict_keys(['users', 'tweets'])

In [56]:
# acessar errors
tweets_recent.errors

[{'resource_id': '1567291270552223744',
  'parameter': 'referenced_tweets.id',
  'resource_type': 'tweet',
  'section': 'includes',
  'title': 'Authorization Error',
  'value': '1567291270552223744',
  'detail': 'Sorry, you are not authorized to see the Tweet with referenced_tweets.id: [1567291270552223744].',
  'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]

In [57]:
# acessar meta
tweets_recent.meta

{'newest_id': '1567301404192120839',
 'oldest_id': '1567263831067398145',
 'result_count': 49,
 'next_token': 'b26v89c19zqg8o3fpz8mu60m1ez3ups0mmlte9kdbn4hp'}

In [60]:
print(f'''
Número de tweets contidos no data: {len(tweets_recent.data)}
Número de intens contidos nos includes de users: {len(tweets_recent.includes['users'])}
Número de intens contidos nos includes de tweets: {len(tweets_recent.includes['tweets'])}
''')


Número de tweets contidos no data: 49
Número de intens contidos nos includes de users: 66
Número de intens contidos nos includes de tweets: 18



In [71]:
tweets_recent.includes['tweets'][0].data

{'conversation_id': '1567184942525779969',
 'geo': {'place_id': '7cd48c674d4c69f7'},
 'lang': 'pt',
 'entities': {'mentions': [{'start': 0,
    'end': 13,
    'username': 'LAMI01773671',
    'id': '1238884889124122626'}]},
 'text': '@LAMI01773671 tudo bem eu já desejo as vezes // vc faz?',
 'in_reply_to_user_id': '1238884889124122626',
 'author_id': '1266799817096904705',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 1,
  'like_count': 1,
  'quote_count': 0},
 'id': '1567210368107253761',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1567204149263048705'}],
 'created_at': '2022-09-06T17:57:23.000Z'}

### Utilizando Paginação para Coleta de Mais Tweets

In [88]:
tweets = []

for tweet in tw.Paginator(
    client.search_recent_tweets,
    query='programação', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency)
    expansions=['author_id', 'geo.place_id', 'entities.mentions.username', 'referenced_tweets.id', 'in_reply_to_user_id', 'referenced_tweets.id.author_id'],
    place_fields=['full_name', 'country'],
    tweet_fields=['id', 'text', 'author_id', 'conversation_id', 'created_at', 'entities', 'in_reply_to_user_id', 'lang',
                   'public_metrics', 'referenced_tweets']).flatten(limit=150):
    tweets.append(tweet)

len(tweets)

150

In [100]:
tweets[149].data

{'id': '1567298886020042754',
 'entities': {'annotations': [{'start': 43,
    'end': 47,
    'probability': 0.343,
    'type': 'Place',
    'normalized_text': 'Verão'},
   {'start': 192,
    'end': 196,
    'probability': 0.6824,
    'type': 'Organization',
    'normalized_text': 'Globo'}],
  'mentions': [{'start': 0,
    'end': 13,
    'username': 'JornalOGlobo',
    'id': '54341363'}]},
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': '@JornalOGlobo Se voltarem com o Horário de Verão, espero que coloquem no país inteiro e não somente no sul, sudeste e centro-oeste.\n\nTinha pavor desse fuso por causa da programação gravada da Globo',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1567120567282843648'}],
 'author_id': '1110925601102352384',
 'lang': 'pt',
 'conversation_id': '1567120567282843648',
 'in_reply_to_user_id': '54341363',
 'created_at': '2022-09-06T23:49:08.000Z'}